Lineare Regression
---
Als ersten Anwendungsfall wird ein simples Maschinellen Lernen Verfahren betrachtet, die Lineare Regression. Hierbei wird ein mathematisches Modell durch Trainingsdaten erstellt, welches verwendet werden kann um ein ausgewähltes Feature vorherzusagen. Ein überschaubarer Datensatz für diese Aufgabe wäre zum Beispiel der Iris Datensatz. Dieser beinhaltet Features von drei verschiedenen Blumenarten, welche wiederum durch eine lineare Regression bestimmt werden können.
<br />
Zunächst muss dafür der Datensatz geladen und in die Trainings- und Testmenge aufgeteilt werden. Die beiden Mengen stehen dabei in einem 80% Trainingsdaten - 20% Testdaten Verhältnis.

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

Bibliotheken Beispiel
---
Um einen Eindruck für eine Lineare Regression auf dem Iris Datensatz zu erhalten wird zuerst die Sklearn Implementierung ausprobiert. Die berechneten Abweichungen von den Ergebnissen können wiederum verglichen werden mit weiteren Tests.

In [2]:
lr = LinearRegression()

lr.fit(X_train, y_train)

lr.predict(X_test)
pred = lr.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
print('Mean Squared Error:', mean_squared_error(y_test, pred))
print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

Mean Absolute Error: 0.1584601230068247
Mean Squared Error: 0.043573599004634006
Mean Root Squared Error: 0.20874290168682144


From Scratch
---
Da die Implementierung aus Sklearn nicht jeden Datentyp unterstützt muss die Lineare Regression selbst implementiert werden. Der Grund dafür liegt in der homomorphen Verschlüsselung. Sobald eine beliebige Zahl verschlüsselt wurde, ist diese keine gewöhnliche Zahl mehr sonder ein verschlüsseltes Objekt und die meisten Bibliotheken erlauben als Parameter nur Zahlen beziehungsweise primitive Datentypen.
<br />
Für die Lineare Regression werden zwei Funktionen implementiert:
* Fit: In dieser Funktion werden die Gewichte w in der Lineare Regression basierend auf den Trainingsdaten erstellt. Dies geschieht indem immer wieder mit den Gewichten Vorhersagen erstellt werden und verändert werden bis diese konvergieren.
* Predict: In dieser Funktion wird die Vorhersage mit einer beliebigen Eingabe und den in der Predict Funktion berechneten Gewichte erzeugt.

In [3]:
def fit(x, y):
	regression = np.c_[x, np.ones(len(x))]

	weights = np.ones(regression.shape[1])

	norma = 1
	learning_rate = 0.00001
	epsilon = 0.9
	while(norma > epsilon):
		y_pred = regression @ weights.T
		dw = regression.T @ (y - y_pred)
		norma = np.sum(np.sqrt(np.square(dw)))

		weights = weights.T + (learning_rate * dw)
	return weights

def predict(w, x):
	return w[:-1] @ (np.array(x).T) + w[-1]

Nachdem die Funktionen implementiert sind können die Gewichte erstellt werden mit den Trainingsdaten. Sobald diese berechnet sind wird ein Feature von den Testdaten vorhersagt. Dabei wird noch die Laufzeit der Vorhersage gemessen um später mit den anderen Tests verglichen zu werden.
<br />
Die erstellten Gewichte können in den Tests mit der homomorphen Verschlüsselung wieder verwendet werden.

In [4]:
weights = fit(X_train, y_train)

pred = predict(weights, X_test)

def meanError(y_test, pred):
	print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
	print('Mean Squared Error:', mean_squared_error(y_test, pred))
	print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred)))

meanError(y_test, pred)

Mean Absolute Error: 0.15791782248674335
Mean Squared Error: 0.042219357807520355
Mean Root Squared Error: 0.2054734966060595


Die berechneten Abweichung zwischen den Vorhersagen und den tatsächlichen Features sind ähnlich zu den Ergebnissen der Sklearn Implementierung und weisen erst ab der 3. Nachkommastelle Unterschiede auf. Die eigene Implementierung scheint dadurch korrekt zu funktionieren.

Vorhersagen auf verschlüsselten Daten
===
In dem folgenden Beispiel soll das trainierte Modell verwendet werden, um die Inferenz auf verschlüsselten Daten durchzuführen. Hierfür wird der Testdatensatz verschlüsselt.

In [5]:
import tenseal as ts

poly_mod_degree = 4096
coeff_mod_bit_sizes = [40, 20, 40]
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_eval.global_scale = 2 ** 20
ctx_eval.generate_galois_keys()

e_text = np.array([[ts.ckks_vector(ctx_eval, [feature]) for feature in item] for item in X_test])

e_result = predict(weights, e_text)

de_result = [result.decrypt() for result in e_result]

meanError(y_test, de_result)

Mean Absolute Error: 0.15665889248734516
Mean Squared Error: 0.04202577492066002
Mean Root Squared Error: 0.2050018900416775
